In [1]:
import os
print(os.getcwd())

/home/broilo/Documents/GitHub/Dataset/UFC-project/data-mining/instagram/Selenium/Khabib-Nurmagomedov/khabib_nurmagomedov


In [11]:
path = '/home/broilo/Documents/GitHub/Dataset/UFC-project/data-mining/instagram/Selenium/Khabib-Nurmagomedov/khabib_nurmagomedov/'

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [5]:
#!pip install selenium

In [4]:
username = 'khabib_nurmagomedov'
browser = webdriver.Chrome('/usr/bin/chromedriver')
browser.get('https://www.instagram.com/'+username+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [5]:
#Extract links from user profile page
links = ['https://www.instagram.com/p/CD6ic6bscKo/']
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
script = body.find('script', text = lambda t: t.startswith('window._sharedData'))
page_json = script.text.split(' = ', 1)[1].rstrip(';')
data = json.loads(page_json)
#try 'script.string' instead of script.text if you get error on index out of range
for link in data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']:
    links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')
#try with ['display_url'] instead of ['shortcode'] if you don't get links 

In [6]:
link

{'node': {'__typename': 'GraphImage',
  'id': '2370502025951759679',
  'shortcode': 'CDltwh8Ms0_',
  'dimensions': {'height': 1080, 'width': 1080},
  'display_url': 'https://instagram.fpet1-1.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/117077353_1662910650533960_8533852559924300796_n.jpg?_nc_ht=instagram.fpet1-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=CchOnXClqC0AX_ppArv&oh=d37444bd02253af9996e10bf09c76ee8&oe=5F67BA80',
  'edge_media_to_tagged_user': {'edges': []},
  'fact_check_overall_rating': None,
  'fact_check_information': None,
  'gating_info': None,
  'media_overlay_info': None,
  'media_preview': 'ACoq06XFRCVP7w/MUyW7jhXexyBjpyeau5FixikNKHVhkEYPvSE07iG0lIWpN1AiZbcDqc/hWZqBw2wY2gDI4GT15xzxWzmsTWlLGMA4+9/SsdzbYdBcZVlbBY4I9QOQfp0yPY1KsrL6H681m2SkLnJx2B/X/Jq905FUIvJcI3DAKfpxU+V9qyKMClYDbxisLV5gwCgjKk59e3T2rbfpWDqajcTgZ+X+VJb2GMtnygX+6MZ7VaFNAAGBxThVCFXAPzcipfsrnpgjsc9ahNJk1Lv0A//Z',
  'owner': {'id': '271229086', 'username': 'khabib_nurmagomedov'},
  'is_video': False,
  'accessibility_cap

In [7]:
result = pd.DataFrame()
for i in range(len(links)):
 try:
     page = urlopen(links[i]).read()
     data = bs(page, 'html.parser')
     body = data.find('body')
     script = body.find('script')
     raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
     json_data = json.loads(raw)
     posts = json_data['entry_data']['PostPage'][0]['graphql']
     posts= json.dumps(posts)
     posts = json.loads(posts)
     x = pd.DataFrame.from_dict(json_normalize(posts), orient = 'columns') 
     x.columns = x.columns.str.replace('shortcode_media.', '')
     result = result.append(x)
 except:
     np.nan
result = result.drop_duplicates(subset = "shortcode")
result.index = range(len(result.index))

In [8]:
result

,__typename,id,shortcode,dimensions.height,dimensions.width,gating_info,fact_check_overall_rating,fact_check_information,sensitivity_friction_info,media_overlay_info,...,video_url,video_view_count,video_play_count,encoding_status,is_published,product_type,title,video_duration,thumbnail_src,clips_music_attribution_info
0,GraphSidecar,2376363273859744424,CD6ic6bscKo,1075,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GraphVideo,2378577252291676775,CECZ2ihq4pn,640,640,None,None,None,None,None,...,https://instagram.fpet1-1.fna.fbcdn.net/v/t50....,1128253.0,None,None,True,feed,,59.466,https://instagram.fpet1-1.fna.fbcdn.net/v/t51....,None
2,GraphVideo,2377674489214847785,CD_Mlm5KMsp,391,750,None,None,None,None,None,...,https://instagram.fpet1-1.fna.fbcdn.net/v/t50....,707701.0,None,None,True,feed,,38.266,https://instagram.fpet1-1.fna.fbcdn.net/v/t51....,None
3,GraphSidecar,2376942120292559334,CD8mEPLs6nm,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GraphVideo,2376448722158205999,CD614WXK7wv,480,480,None,None,None,None,None,...,https://instagram.fpet1-1.fna.fbcdn.net/v/t50....,1169440.0,None,None,True,feed,,58.866,https://instagram.fpet1-1.fna.fbcdn.net/v/t51....,None
5,GraphSidecar,2375534336120413683,CD3l-SAM13z,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GraphSidecar,2374775491567742371,CD05bo8Mu2j,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GraphImage,2374087329014877735,CDyc9jnsA4n,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GraphImage,2373691171541192729,CDxC4tOMZAZ,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,GraphImage,2372938936339395514,CDuX2PmM_O6,1080,1080,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
import os
import requests
result.index = range(len(result.index))
directory = path
for i in range(len(result)):
    r = requests.get(result['display_url'][i])
    with open(directory+result['shortcode'][i]+".jpg", 'wb') as f:
                    f.write(r.content)

In [10]:
result.shortcode

0     CD6ic6bscKo
1     CECZ2ihq4pn
2     CD_Mlm5KMsp
3     CD8mEPLs6nm
4     CD614WXK7wv
5     CD3l-SAM13z
6     CD05bo8Mu2j
7     CDyc9jnsA4n
8     CDxC4tOMZAZ
9     CDuX2PmM_O6
10    CDqrDaQsKej
11    CDltwh8Ms0_
Name: shortcode, dtype: object